In [1]:
from receive_func import*

Loading BokehJS ...

In [2]:
sd.query_devices()

  0 bcm2835 ALSA: - (hw:0,0), ALSA (0 in, 2 out)
  1 bcm2835 ALSA: IEC958/HDMI (hw:0,1), ALSA (0 in, 2 out)
  2 Fe-Pi Audio: - (hw:1,0), ALSA (2 in, 2 out)
  3 sysdefault, ALSA (0 in, 128 out)
  4 dmix, ALSA (0 in, 2 out)
* 5 default, ALSA (2 in, 2 out)

In [3]:
builtin_idx = 0
usb_idx = 2
sd.default.samplerate=48000
sd.default.channels = 1

In [41]:
def queueREPLAY_callback(indata,outdata, frames, time, status):
    if status:
        print(status)
    outdata[:] = indata
    Qin.put( indata.copy()[:,0] )  # Global queue
    
st_replay = sd.Stream( device=(usb_idx),callback=queueREPLAY_callback)
Qin = Queue.Queue()
print("Starting streams")

fs = 48000
modem = TNCaprs(fs = fs ,Abuffer = 512,Nchunks = 10,baud=2400,center=2400)

st_replay.start()
time.sleep(5)
starttime = time.time()

fname = "recon.gz"
f = open(fname,"wb")
flag=1
# for you to complete
npack = 0
packetnumber = 1
while (1):
    packets  = modem.processBuffer(Qin.get()) 
    ax = modem.decodeAX25(packets)
    for pkt in packets: 
        npack = npack + 1
        ax = modem.decodeAX25(pkt)
        infostr = "%(n) 2i) | DEST: %(dest)s | SRC: %(src)s | DIGI: %(digi)s | %(info)s |" % {
                        'n': npack,
                        'dest': ax.destination,
                        'src': ax.source,
                        'digi': ax.digipeaters,
                        'info': ax.info.strip()
                    }
        print(infostr)
        f.write(ax.info)
    if (npack == 1) and (flag==1):
#         print(ax.destination.split())
        temp = ax.destination.split()[0]
#         print(temp)
        packetnumber = int(temp)
        flag=0
#         print(packetnumber)
    if npack == packetnumber:
        break

st_replay.stop()
st_replay.close()
f.close()

print("Done")
print(time.time() - starttime)

Starting streams
 1) | DEST: 34    0 | SRC: KN6AHX | DIGI:       0 | b'\x1f\x8b\x08\x08\x94V\xd2\\\x02\xffdata\x00\xed]\t\xb3#Wu\x1e\x9b@\xc8\x04\xc8B62$\x98\x04\xe2\x17\x87\x10L<\xd8\xb3yf<\x13B2\xc1\x90\x97\xbc\xaa\xd6{O\xfb\xd2\x8b\xd4\xcb\xedE\xfbSl\xe3\x19\xdb\xa4\x80\xaa\xd1\xdb\xa0\xa0\x9cP\xe5LL\xa8\xcaZY\xc0K\x95\xf4\xc3\x92\xef\xdc\xdb\xddj\xb5ZRKO\x03\xd9\xf4\xae\x9bV\xab\xfb\xf6\xf9\xce\xf2\x9ds\xaeZ\xc3\xf0\xf9\x9d/~9\xf3\xc8\xb9\xcf\x9f\xeb=^\xa9:e\xfb\xf1\xcb\x8f=\xde\xff\xd2\xe3\x9fz\xec\xf1\x9ai\xbbv\xd1\xc8\x9bv\xa5J\x87?_l8U\x1cw\x94\xa2U\xc5\xfb\xad\xdf\xff\xd4o\x7f\xea\xb1\x83\xc7\xa2\xaf\xf3/<Z6<\xdd\xea|\xbal\xda\xd5O\xeb^\xc3U\x8b\xb6]\xec\x9c\xcf\xdb\xd5\xb2i8\xae\xed\x95\xdd\xf3\xec\x9c8\xeb\xbcQ\x11\x9f\xb2G\xee\x9c\xbb\xcb\x1e-\xe7\xcbf\xa5Zv\xceW\r\xda9\xcf\xde#=r\xee\xdc\xb9\x12\xfb\t\xe9' |
 2) | DEST: 34    0 | SRC: KN6AHX | DIGI:       0 | b'}\xf8\xdfF\xd1U\x8d\'\xd9{\xef\xb1\xf7m\xb3\x9f|\x85\xbd\x7f\x9b\xfd\xd4\xd6\x9dG\xee\xbc\xe7.;\xefOXq;\x16\xae\x

 12) | DEST: 34    0 | SRC: KN6AHX | DIGI:       0 | b'hF\xae\xac\xec\x97P\x9f\x94\x8b\x15\xc4\x99\xddj\xba\xed\x16.\xf5z\x94\x0f\xa9\x16r\t\x05c.h\x1f3\xe0\xf6\xcdf\x93\xb08.P\x91\xb9\x14MW\x1aT\x1bCyP\xbe\x90?\x1cp\x1e\xd4\x86v\xb3ey0\xa8\xab\x19VU\x85_tDZ\x87p\x80\x83\x0e\x0bp\x80B\xae\x9b\xba\xdd\xd4\x90ht\x1b(4t\x98t3h\xcf\xe3h\x9ax\xcbPS2\xdb\xb4\x18\x86a\xa1\xbcD:D\xcb\x83\xde\xcd(\xd7@\x172\x8e@~\x9c\x8bm\xa7\xdd\xa4;tH\x0fTf\xb6[\xa6Cs\xc2\xbe\xa8\xd7\xf0\xb6\xd5n\x02\x08nB~\xe72\x8dT\x87\xfa\xb0/\xfa\x05\x1f\xc5\x80\x03\xe9\xf7\t\x88\xd7\x84A\xe9\x8e\r\xf4\xb8-\xcb\xc5\x0c\x1dr\x97\x1e\x95\xc1T@v\xba\xc0\x82\xfa\x07\xc9\x05[\x0cd+\xa4\x0f\x07\xb3\xdb6\x84\xc7\xcb\xe6\xc30!\xbc\xc9\xb7\x16\xc1a\x80c\x9b\xcc\xae\x1bhc\x1bZ\xc3\xb4\x1d\x1b\xb6\x83\xb4P|\x1b\x16\x81\xc8@\xc1' |
 13) | DEST: 34    0 | SRC: KN6AHX | DIGI:       0 | b'\xad\x83\x81\x8fZm\xaa\x9d\xb1\x0f\xf9\x1d\x92\xdfv\x1d\xda6=G3lX\x82\x80\x08\x8f\x1a\x0c\xc2H\xc7;\x88\xec\xb8\x1e\xee\xd5\xc0\xadAf-z\xcb!t\xa9\xd9\

 23) | DEST: 34    0 | SRC: KN6AHX | DIGI:       0 | b'tU\xc82\x97z\x87;\x19D\xc7VN>Q#h\xfc\x05\xaf\xc3\x16\x11\x98\xdeg\xe8\x82\x8b;\x81W\xd3;\xffS\xca}\xc7;\x11SrI\x14\xe8\xe1\xc2\xb5\x9d\xb3\xf9\xeaZ\xce-\xfe \xc1\xe2\x93w\xc2{\x00L`\xef\xd7\xe3NG_Y\xf9\x86\xf6U\'\xab;;\x9b\x88C":\xf2\x8b4g+\x83\x80\x93\x96\x84\xc4\xd3\x97\'\\\x0b1\x05\xa6\xc0f;\x11W\x9cL\xa2\x9e\r\xc8N0\xc3&\xb4\xb2\x90\x86\xe0t\xd0\x96\xefw\x17.\x9c\x9b\xa3\xd47C?x7\xea\x0f\xc4\n+\xf2\x89<\xe3-\xe2\xc8\x8e<?wL\xf4\xad\xccs\xe1\x98_>\xc9\x01\tX\x17\x82\x00[\xc0};\x01\x99\nE\xc0\xcd\xd5uM\x88\x1e\x89G\xca2\xdb\x81\xdc\xbe\xa2\tOE\xa6\xfe\xcb\xf1]?\xee\xdf\x9c\xca\x94\x0b\xae\x1f.\xf1!~{Y\x8d\x90\xe1\xce\xba\xee\xa4\xa4\x0c\xbe\xf1\x8b\xd3w\x9f/\xdbN\xd4\xb98\xd3\xbf\n+\xbf\xc6\r\xb4\x98d\xf8\xe9C\xdf' |
 24) | DEST: 34    0 | SRC: KN6AHX | DIGI:       0 | b'\x8e\xeb\xe4\xf0D7\xb81_\xe0)\xe6\x1aN\x1d\xe2y\'\xc1(\x02\xdf\x0e/\xbb\x8e\xb8M\xc5\x1d9\xc3_\x10\x04\xf2\xc8\xa3\x8fF\xaf\x0c\xfdM\xe0\x9bU\xd5\xe6)>\x9c\xfa\x

 34) | DEST: 34    0 | SRC: KN6AHX | DIGI:       0 | b"t\x1c\x00\xb9\xb5us\xaa\x1b^\xc7\x15S-\x99\x05\\\xcd\x1f\xb4\x08\xc2+\x04x:\xfe\x16\xff\xf0]\x1f\xe6a2\xc6\xa5k\xca\xa2\x98\r=U\xd5\x06\xb7\xaf\xf3\x05\x00\xde1\x8au\xb6\xe1*\x81\xb7uM\xe2+_\xc7\xfesZ\x87\xc9\xb5\xd2 \xbaT-b\xcc\xff\xef\xc4\xafq^\xe2f|\x8do_\x9c|\xdd\x95\x04s\xf9\xc3+J\x82\x8b\xc3\x94\xcf\x88\xa7\xa7Dx\xac\xfc\xf8\x10\xcaI\xf6'\xca{\xef\xb1/n\xb3\xe7]\xf6\xa5R5\xcb\xbe\xbc\xb5u\xe7\x03w~\xe3\xce\xad;\xefq\xd9\x9f\xde\xb9\xfb\xc5\xf3\x8f\xdcc\xdb\xd5\xaa\xcb\xfe\xac\xf4\xe9\xff\x02\xf9\x88\xc4\x03\x04\x91\x00\x00" |
Done
67.41734766960144


In [33]:

st_replay.stop()
st_replay.close()
f.close()


In [42]:
f = gzip.GzipFile("recon.gz", "r")
data = np.load(f)
f.close()

In [43]:
print(data)

[ [array([ 32317.86071655,   5172.73887652,   3340.82669233,   2364.90063311,
         1992.49570458]), [216.90439178211889, 1386.7811148632704], [-0.56945209215743309, -0.091596391398765117]]
 [array([[ 59,  62, 206, 124, 153],
       [ 59,  63, 206, 125, 152],
       [ 59,  72, 200, 122, 153],
       [ 60, 160, 172,   2,  17],
       [ 60, 194, 156, 176,  99],
       [ 60, 197, 130, 175, 126],
       [ 60, 198, 111, 168, 151],
       [ 61, 198,  93, 161, 166],
       [ 61, 157,  55,   0, 120],
       [ 63,  75,  37,  71, 255],
       [ 60,  53,  45, 172,  49],
       [ 61,  48,  55, 177,  38]], dtype=uint8), array([[105, 108, 113, ..., 113, 114, 113],
       [150, 125, 125, ..., 202, 189, 167],
       [126, 126, 126, ..., 126, 126, 126],
       [126, 126, 126, ..., 167, 167, 126],
       [126, 126, 126, ...,  88,  88, 104]], dtype=uint8)]
 [(12, 34, 67, 3), (133, 266)]]


In [44]:
img_rec = recon(data)

In [45]:
img = imageStack_load(filename = 'EE123_Final_Video.png')
# print(time.time()-starttime)

In [46]:
recon_display('final_rec',img_rec,img)

PSNR:  18.4660087065
